## Transfer Learning
Use pre-trained backbone from `run_baseline_pretrain_moco.ipynb`

### Environment Setup

In [ ]:
# import libraries
import os
import sys
import argparse
from tqdm import tqdm

import numpy as np
import seaborn as sns
import pandas as pd
import math
from copy import deepcopy

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display

import pickle
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Collab Needs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Define filepaths to required input (scripts, data) and outputs

In [ ]:
# ----------------------------------------------------
# Inputs root
# ----------------------------------------------------
FP_ROOT="/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/"

# ----------------------------------------------------
# Dataset info
# ----------------------------------------------------
# Images zip files
NIH_DATASET_PATH_SPLIT = FP_ROOT + "Data/NIH_Chest_XR_Pneumonia.zip"
# Chexpert
DATASET_PATH_SPLIT = FP_ROOT + "Data/CheXpert_reduced_dataset_split.zip"

# Labels
# NIH
TRAIN_LABELS_CSV = FP_ROOT + "Data/nih_train.csv"
VAL_LABELS_CSV = FP_ROOT + "Data/nih_val.csv"
TEST_LABELS_CSV = FP_ROOT + "Data/nih_test.csv"

# ChexPert
# TRAIN_LABELS_CSV = FP_ROOT + "Data/final_project_updated_names_train.csv"
# VAL_LABELS_CSV = FP_ROOT + "Data/final_project_updated_names_val.csv"
# TEST_LABELS_CSV = FP_ROOT + "Data/final_project_updated_names_test.csv"

# ----------------------------------------------------
# Model SRC
# ----------------------------------------------------
SRC_ROOT = FP_ROOT + "src/"
TRAIN_SCRIPT = f"{SRC_ROOT}/train_moco.py"
MOCO_FOLDER = f"{SRC_ROOT}/moco"

# ----------------------------------------------------
# Outputs
# ----------------------------------------------------
ROOT_ARTIFACT_SAVE = FP_ROOT + "artifacts/"

In [ ]:
# Add the project /src to the system path
if SRC_ROOT not in sys.path:
    sys.path.append(SRC_ROOT)
    print(f"Added {SRC_ROOT} to sys.path")

print(sys.path)

## Unzip data

NIH

In [ ]:
# Unzip the dataset (image) files to /tmp
DATA_DEST_UNZIPPED = "/tmp/NIH_Chest/"
os.makedirs(DATA_DEST_UNZIPPED, exist_ok=True)
!unzip "{NIH_DATASET_PATH_SPLIT}" -d {DATA_DEST_UNZIPPED} # need to use "" to accomdate space

In [ ]:
# Update for the unzipped sub-name
DATA_DEST_UNZIPPED = "/tmp/NIH_Chest/NIH_Chest_XR_Pneumonia"

# Train subset
DATA_DEST_UNZIPPED_TRAIN = "/tmp/NIH_Chest/NIH_Chest_XR_Pneumonia/train"

## 2) Fine tune the classifier

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src"

In [ ]:
# Load in the pre-trained baseline MoCo model

from moco.model_builder import MoCo

# Instantiate the MoCo model with updated momentum
moco_model = MoCo(dim=128, K=65536, m=0.999, T=0.2, pretrained=True, device='cuda')

# Extract the checkpoint state dict
MOCO_BACKBONE_SAVE_PATH = f"{ROOT_ARTIFACT_SAVE}/moco_checkpoint_epoch_10.pth"
#checkpoint = torch.load(MOCO_BACKBONE_SAVE_PATH, map_location=device)
#moco_model.load_state_dict(checkpoint['model_state'])

### Run finetuning using the finetune_resnet module

In [ ]:
ROOT_ARTIFACT_SAVE

In [ ]:
! python finetune_resnet.py \
    --train_csv "$TRAIN_LABELS_CSV" \
    --val_csv "$VAL_LABELS_CSV" \
    --test_csv "$TEST_LABELS_CSV" \
    --root_dir "$DATA_DEST_UNZIPPED" \
    --pretrained_encoder "$MOCO_BACKBONE_SAVE_PATH" \
    --artifact_root "$ROOT_ARTIFACT_SAVE" \
    --n_epochs 8